In [129]:
import math
import sys
from collections import namedtuple, deque
from pprint import pprint as pp

In [144]:
class Graph:
    def __init__(self, gdict = {}):
        self.gdict = gdict
        self.colour = {}
        self.distance = {}
        self.predecessor = {}
        self.finish = {}     
        
    def get_vertices(self):
        #return list(self.gdict.keys())
        vertices = set()
        for u in self.gdict:
            vertices.add(u)
            for v in self.gdict[u]:
                vertices.add(v)
        return vertices
    
    def get_edges(self):
        edgename = []
        for vertex in self.gdict:
            for next_vertex in self.gdict[vertex]:
                if {next_vertex, vertex} not in edgename:
                    edgename.append({vertex, next_vertex})
        return edgename    
    
    def add_vertex(self, vertex):
        if vertex not in self.gdict:
            self.gdict[vertex] = []
            
    def add_edge(self, edge):
        edge = set(edge)
        (vertex1, vertex2) = tuple(edge)
        if vertex1 in self.gdict:
            self.gdict[vertex1].append(vertex2)
        else:
            self.gdict[vertex1] = [vertex2]
            
    # BFS and DFS are implemented using the pseudo-codes from the Book
    def BFS(self, s):
        vertices = self.get_vertices()
        vertices.remove(s)
        for u in vertices:
            self.colour[u] = "White"
            self.distance[u] = 0
            self.predecessor[u] = None
        self.colour[s] = "Gray"
        self.distance[s] = 0
        self.predecessor[s] = None
        queue = []
        queue.append(s)
        while queue:
            u = queue.pop(0)
            #print(self.gdict[u])
            for v in self.gdict[u]:
                if self.colour[v] == "White":
                    self.colour[v] = "Gray"
                    self.distance[v] = self.distance[u] + 1
                    self.predecessor[v] = u
                    queue.append(v)
            self.colour[u] = "Black"    
            
    def print_path(self, s, v):
        if v in self.gdict.keys():
            if v == s:
                print(s)
            elif self.predecessor[v] == None:
                print("There is no path from", s, "to", v, "exists.")
            else:
                self.print_path(s, self.predecessor[v])
                print(v)
        else: 
            print("Node with key", v, "is not in the graph.")       
            
    def DFS(self):
        for u in self.gdict:
            self.colour[u] = "White"
            self.predecessor[u] = None
            self.distance[u] = 0    # Added to initialise self.distance
            self.finish[u] = -1     # Added to initialise self.finish
        global time 
        time = 0
        for u in self.gdict:
            if self.colour[u] == "White":
                self._DFS_visit(u)        
                
    def _DFS_visit(self, u):
        global time
        time += 1      # white vertex u has just been discovered
        self.distance[u] = time
        self.colour[u] = "Gray"
        for v in self.gdict[u]:  # explore edge(u,v)
            if self.colour[v] == "White":
                self.predecessor[v] = u
                self._DFS_visit(v)
        self.colour[u] = "Black"   # blacken u; it is finished
        time += 1
        self.finish[u] = time

    # ----- New functionality -----

    def initialise_single_source(self, s):
        for v in self.get_vertices():
            self.distance[v] = math.inf
            self.predecessor[v] = None       
        self.distance[s] = 0

    def get_weight(self, u, v):
        return self.gdict[u][v]

    def relax(self, u, v):
        if self.distance[v] > self.distance[u] + self.get_weight(u, v):
            self.distance[v] = self.distance[u] + self.get_weight(u, v)
            self.predecessor[v] = u

    # DAG
    def dag_shortest_paths(self, s):
        self.topological_sort()  # O(V+E)
        self.initialise_single_source(s)  # O(V)
        for u in self.finish.keys():      # O(V+E)
            for v in self.gdict[u].keys():
                self.relax(u, v)
        
    
    def extract_min(self, candidates):
        min = 999999
        for cu in candidates:
            if self.distance[cu] < min:
                min = self.distance[cu]
                u = cu  
        return u
    
    def bellman_ford(self, s):
        self.initialise_single_source(s)       # O(V)
        for i in range(1, len(self.get_vertices())):    # O(E)
            for (u, v) in self.get_edges():
                self.relax(u, v)
        for (u,v) in self.get_edges():    # O(E)
            if self.distance[v] > self.distance[u] + self.get_weight(u, v):
                return False
        return True

    def topological_sort(self):
        self.DFS()
        #return self.finish()


        
     # Dijkstra Algorithm
    def dijkstra(self, s):
        self.initialise_single_source(s)
        S = [] # S is a list for vertices that have been processed
        Q = self.get_vertices()  # Q is a list for vertices that have not been processed
        u = None
        while Q:
            u = self.extract_min(Q)
            Q.remove(u)
            S = S + [u] 
            for v in self.gdict[u]:
                self.relax(u, v)
                
    def dijkstra2(self, s):
        self.initialise_single_source(s)
        dist_far = [] # distance
        Q = self.get_vertices() # All verticies
        while Q:
            u = self.extract_min(Q)
            Q.remove(u)
            dist_far = dist_far + [u] 
            for v in self.gdict[u]:
                self.relax(u, v)
    


In [145]:
adjacency5 = {"s" : {"t": 10, "y": 5},
              "t" : {"x": 1, "y": 2},
              "x" : {"z": 4},
              "y" : {"t": 3, "x": 9, "z": 2},
              "z" : {"x": 6, "s": 7}
              }

g5 = Graph(adjacency5)

In [146]:
g5.dijkstra2("s")

In [164]:
g5.print_path("s", "z")

s
y
z


In [122]:
g5.print_path("s", "x")

s
y
t
x


In [123]:
g5.dag_shortest_paths("s")

In [125]:
g5.print_path("s", "z")

s
y
z


In [105]:
g5.dag_shortest_paths('s')

In [106]:
adjacency4 = { "r" : {"s": 5, "t": 3},
              "s" : {"t": 2, "x": 6},
              "t" : {"x": 7, "y": 4, "z": 2},
              "x" : {"y": -1, "z": 1},
              "y" : {"z": -2},
              "z" : {}
             }

g4 = Graph(adjacency4)

In [98]:
g4.dag_shortest_paths("s")

In [99]:
g4.print_path("s", "z")

s
x
y
z


In [ ]:
    # Dijkstra Algorithm
    def dijkstra1(self, s):
        #pri = pq()
        self.initialise_single_source(s)
        # S is a list for vertices that have been processed
        S = []
        # Q is a list for vertices that have not been processed
        Q = self.get_vertices() 
        while Q:
            u = self.extract_min(Q)
            Q.remove(u)
            S = S + [u] 
            #pri.insert(u)
            for v in self.gdict[u]:
                self.relax(u, v)
                
        cc = self.predecessor
        dest = []
        while len(self.predecessor):
            dest = u
            u = self.predecessor[u]
        dest = u

In [128]:
def dijkstra1(self, s):
        #pri = pq()
        self.initialise_single_source(s)
        # S is a list for vertices that have been processed
        S = []
        # Q is a list for vertices that have not been processed
        Q = self.get_vertices() 
        while Q:
            u = self.extract_min(Q)
            Q.remove(u)
            S = S + [u] 
            #pri.insert(u)
            for v in self.gdict[u]:
                self.relax(u, v)
                
        cc = self.predecessor
        dest = []
        while len(self.predecessor):
            dest = u
            u = self.predecessor[u]
        dest = u
        
# -----------------------------------
def dijkstra2(self, s):
    self.initialise_single_source(s)
    dist_far = [] # distance
    dist_far[s] = 0
    Q = self.get_vertices() # All verticies
    while Q:
        u = self.extract_min(Q)
        Q.remove(u)
        dist_far = dist_far + [u] 
        for v in self.gdict[u]:
            self.relax(u, v)

## Problem 2

In [1]:
import os

files = [f for f in os.listdir('./Data') if os.path.isfile(f)]
for f_name in files:
    with open(f_name, "r") as f:
        tram_line = f.readlines()
        print(tram_line)

In [2]:
files

[]

In [28]:
import os

# Get .txt files
listn = []
for f_name in os.listdir('Data'):
    if f_name.endswith('.txt'):
        listn.append(f_name)
        #print(f_name)
print(listn)

['tram1.txt', 'tram10.txt', 'tram11.txt', 'tram14.txt', 'tram2.txt', 'tram3.txt', 'tram4.txt', 'tram5.txt', 'tram6.txt', 'tram7.txt', 'tram8.txt', 'tram9.txt']


In [36]:
i = 0
lis = []
files = [f for f in os.listdir('./Data/') if os.path.isfile(f)]

for f_name in files:
    with open(f_name, "r", encoding="utf-8") as f:
        tram_line = f.readlines()
        lis.append(f.readlines())
        i += 1
        lis += tram_line
        #print(tram_line)

lis

[[],
 'Opaltorget, 1\n',
 'Smaragdgatan, 1\n',
 'Briljantgatan, 1\n',
 'Frölunda Torg Spårvagn, 1\n',
 'Positivgatan, 1\n',
 'Musikvägen, 1\n',
 'Nymilsgatan, 1\n',
 'Lantmilsgatan, 2\n',
 'Axel Dahlströms Torg, 1\n',
 'Marklandsgatan, 3\n',
 'Botaniska Trädgården, 2\n',
 'Linnéplatsen, 1\n',
 'Olivedalsgatan, 2\n',
 'Prinsgatan, 2\n',
 'Järntorget, 2\n',
 'Stenpiren, 3\n',
 'Brunnsparken, 2\n',
 'Centralstationen, 3\n',
 'Ullevi Norra, 2\n',
 'Svingeln, 2\n',
 'Olskrokstorget, 3\n',
 'Redbergsplatsen, 2\n',
 'Stockholmsgatan, 1\n',
 'Härlanda, 2\n',
 'Munkebäckstorget, 1\n',
 'Ättehögsgatan, 1\n',
 'Kaggeledstorget, 2\n',
 'Tingvallsvägen, 0',
 [],
 'Väderilsgatan, 1\n',
 'Friskväderstorget, 1\n',
 'Önskevädersgatan, 1\n',
 'Mildvädersgatan, 1\n',
 'Vårväderstorget, 2\n',
 'Sälöfjordsgatan, 1\n',
 'Eketrägatan, 1\n',
 'Gropegårdsgatan, 1\n',
 'Rambergsvallen, 1\n',
 'Wieselgrensplatsen, 2\n',
 'Vågmästareplatsen, 1\n',
 'Hjalmar Brantingsplatsen, 1\n',
 'Frihamnen, 4\n',
 'Lilla Bomme

In [4]:
f = open("tram3.txt", "r")
print(f.read())

FileNotFoundError: [Errno 2] No such file or directory: 'tram3.txt'

In [14]:
import os

tram_line = []
files = [f for f in os.listdir('./Data') if os.path.isfile(f)]
print(files)
for f_name in files:
    with open(f_name, "r") as f:
        tram_line.append(f.readlines())
        print(tram_line)
tram_line

[]


[]